Иморт библиотек

In [ ]:
 # data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

 # visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Загрузка таблиц

In [ ]:
train_df = pd.read_csv('train_longevity.csv')
test_df = pd.read_csv('test_longevity.csv')

Рассмотрим вид загруженных таблиц

In [ ]:
display(train_df.head())
train_df.info()
#display(test_df.head())
#test_df.info()

**В датасетах приведена следующая информация о пожилых людях (атрибуты объекта):**

* Longevity - Класс активного долголетия: 1 - человек доживет до 90 лет; 0 - нет
* Id - Идентификатор пожилого человека;
* Education - Образование: 1 - высшее; 2 - среднее; 3 без образования;
* Sex - Пол;
* Age - Возраст;
* Pet - Пожилой человек ухаживает за домашними животными: указано количество;
* Children - Пожилой человек проживают с детьми/внуками/близкими родственниками: указано количество проживающих совместно в пожилым человеком;
* Region - Регион проживания;
* Activity - Уровень физической активности (количество шагов в день): данные получены от специального приложения;
* MedExam - Посещение поликлиники (за послений год): кодирование посещений на основе заполненной медицинской карточки;
* Sport - Физические упражнения: '+' пожилой человек занимается спортом (ходьба, бег, плаванье); '-' не занимается.

Пропущенные значения в поле `Age` не могут быть заполнены - удаляем их и приводим столбец к целочисленному типу.

In [ ]:
# Признак Age должен быть дополнен 
# и фильтрован для обработки алгоритмами.

train_df['Age'].value_counts()
train_df_clean = train_df.dropna(subset=['Age']).reset_index()
train_df_clean = train_df_clean.query('Age != "#ЗНАЧ!"').copy(deep=True)
train_df_clean['Age'] = train_df_clean['Age'].astype('int')

test_df['Age'].value_counts()
test_df_clean = test_df.dropna(subset=['Age']).reset_index()
test_df_clean = test_df_clean.query('Age != "#ЗНАЧ!"').copy(deep=True)
test_df_clean['Age'] = test_df_clean['Age'].astype('int')

Дропаем ненужное, так как корреляция между параметрами и фактом долголетия маловероятна.

In [ ]:
# Produces error. Tables already clean
#train_df_clean = train_df_clean.drop(['MedExam', 'Region', 'Id'], axis=1)
#train_df_clean.head()
#test_df_clean = test_df_clean.drop(['MedExam', 'Region', 'Id'], axis=1)
#test_df_clean.head()

Если у нас нет информации о том, занимается человек спортом или нет, мы заполняем пропущенные значения '-'.

In [ ]:
# Необходимо дополнить функцию Sport, 
# поскольку она также может соотноситься с долголетием.
train_df_clean['Sport'] = train_df_clean['Sport'].fillna(value='-')
#train_df_clean.head()

test_df_clean['Sport'] = test_df_clean['Sport'].fillna(value='-')
test_df_clean.head()

Количество любящих единиц пожилого человека.

In [ ]:
# Cоздать новый признак под названием Family 
# (Семья на основе детей и домашних животных), 
# чтобы получить общее количество членов семьи.
train_df_clean['Family'] = train_df_clean['Children'] + train_df_clean['Pet']
test_df_clean['Family'] = test_df_clean['Children'] + test_df_clean['Pet']

In [ ]:
#sns.barplot(data=train_df_clean, x='Family', y='Longevity')
sns.barplot(data=test_df_clean, x='Family', y='Longevity')

График может говорить о следующем: с ростом количества голов в семье количество долгожителей в группе увеличивается, однако по достижению критического количества социальная нагрузка (и уровень стресса) пожилого человека становится слишком большой - и количество долгожителей начинает уменьшаться. Однако, выводы преждевременные.

Признак: нет никого, кто любит пожилого человека.

In [ ]:
# Заздать новый признак IsAlone, 
# проживающих одиноко без домашних животных и родственников.
train_df_clean['IsAlone'] = (train_df_clean['Family'] == 0)

test_df_clean['IsAlone'] = (test_df_clean['Family'] == 0)

Превращаем количественную переменную в категориальню - вводим порядковый номер диапазона возрастного равновесия.

In [ ]:
# Cоздать новый признак диапазонов возрасного равновесия 
# на основе признака Age, т.к. это поможет разделить пожилых 
# людей на группы условно равновесного состояния 
# (смертность повышается в определенные периоды времени 
# между 70 и 80 годами, и в другие моменты резко снижается) 
# для следующих интервалов: 
# { (...,70](70,72],(72,74],(74,76],(76,78],(80,..)}

def age_range(age):
    if (age <= 70):
        return 1;
    if (age <= 72):
        return 2;
    if (age <= 74):
        return 3;
    if (age <= 76):
        return 4;
    if (age <= 78):
        return 5;
    return 6;
    
train_df_clean['Age_range'] = train_df_clean['Age'].apply(age_range)
test_df_clean['Age_range'] = test_df_clean['Age'].apply(age_range)

In [ ]:
#sns.barplot(data=train_df_clean, x='Age_range', y='Longevity')
sns.barplot(data=test_df_clean, x='Age_range', y='Longevity')

Из графика видно, что количество долгожителей обратно пропорционально порядковому номеру равновесной возрастной группы. Иначе говоря, смертность увеличивается с ростом возраста.

In [ ]:
# Cоздать новый признак для возрастных групп (Age,Education), 
# указывающий на социальный статус пожилого человека. 
# Это превращает непрерывный числовой признак в порядковый 
# категориальный признак.

train_df_clean['Social_status'] = train_df_clean['Age_range'] * train_df_clean['Education']
test_df_clean['Social_status'] = test_df_clean['Age_range'] * test_df_clean['Education']

In [ ]:
#sns.barplot(data=train_df_clean, x='Social_status', y='Longevity')
sns.barplot(data=test_df_clean, x='Social_status', y='Longevity')

Социальный статус - произведение возраста на образование, упорядочивает пожилых людей по снижению дееспособности и снижению квалификации, по типу (дееспособные, квалифицированные), (недееспособные, квалифицированные), (недееспособные, неквалифицированные) c **увеличением** параметра social_status.  
Из графика наблюдается общая тендеция к снижению количества долгожителей с ухудшением социального статуса.

Приведем поле `sport` к целочисленному значению для упрощения дальнейшей работы.

In [ ]:
train_df_clean['Sport'] = (train_df_clean['Sport'] == '+').astype('int')
test_df_clean['Sport'] = (test_df_clean['Sport'] == '+').astype('int')

Рассмотрим итоговую таблицу.

In [ ]:
#display(train_df_clean.head())
#train_df_clean.info()
display(test_df_clean.head())
test_df_clean.info()

Сохраним готовый датафрейм в csv-файл

In [24]:
#file = open("Day2_df.csv", 'w', encoding = 'utf-8', newline = '')
#type(train_df_clean) # Type is pandas.DataFrame; So, we can use the following method
#train_df_clean.to_csv(file)
file = open("Day2_test.csv", 'w', encoding = 'utf-8', newline = '')
test_df_clean.to_csv(file)
test_df_clean.to_csv(file)
file.close()
